In [4]:
import numpy as np
import glob, os, json
import pickle

import matplotlib.pyplot as plt
import corner
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import enterprise
from enterprise.pulsar import Pulsar
import enterprise.signals.parameter as parameter
from enterprise.signals import utils
from enterprise.signals import signal_base
from enterprise.signals import selections
from enterprise.signals.selections import Selection
from enterprise.signals import white_signals
from enterprise.signals import gp_signals
from enterprise.signals import deterministic_signals

from PTMCMCSampler.PTMCMCSampler import PTSampler as ptmcmc

import libstempo as libs
import libstempo.plot as LP, libstempo.toasim as LT

In [2]:
# import the par and tim files
datadir = './data/partim/'

parfiles = sorted(glob.glob(datadir + '*.par'))
timfiles = sorted(glob.glob(datadir + '*.tim'))

psrs = []
for p, t in zip(parfiles, timfiles):
    psrname = parfiles[0].split('/')[-1].split('_')[0]
    psr = Pulsar(p, t, ephem='DE438')
    psrs.append(psr)

In [3]:
for psr in psrs:
    print(psr.name)

B1855+09
B1937+21
B1953+29
J0023+0923
J0030+0451
J0340+4130
J0613-0200
J0636+5128
J0645+5158
J0740+6620
J0931-1902
J1012+5307
J1024-0719
J1125+7819
J1453+1902
J1455-3330
J1600-3053
J1614-2230
J1640+2224
J1643-1224
J1713+0747
J1738+0333
J1741+1351
J1744-1134
J1747-4036
J1832-0836
J1853+1303
J1903+0327
J1909-3744
J1910+1256
J1911+1347
J1918-0642
J1923+2515
J1944+0907
J2010-1323
J2017+0603
J2033+1734
J2043+1711
J2145-0750
J2214+3000
J2229+2643
J2234+0611
J2234+0944
J2302+4442
J2317+1439


In [5]:
## Get parameter noise dictionary
noisedir = os.getcwd() + '/data/noisefiles/'

noise_params = {}
for filepath in glob.iglob(noisedir + '/*.json'):
    with open(filepath, 'r') as f:
        noise_params.update(json.load(f))
# print(noise_params)

In [6]:
print(len(parfiles))

45


In [7]:
# cut down to pulsars that have been measured for 6 years
psrcut = []
for psr in psrs:
    tmax = max(psr.toas)
    tmin = min(psr.toas)
    Tspan = tmax - tmin
    if Tspan / 525600 / 60 > 6:
        psrcut.append(psr)
    # print(psr.name)
print(len(psrcut))


22


In [8]:
pickle.dump(psrcut, open("psrs.p", "wb"))

In [9]:
psrcut = pickle.load(open("psrs.p", "rb"))

In [13]:
print(len(psrcut))

22


In [11]:
# import each pulsar into tempo2
tempopsr = []
for psr in psrcut:
    parfiles = sorted(glob.glob(datadir + psr.name + '*.par'))
    timfiles = sorted(glob.glob(datadir + psr.name + '*.tim'))
    psr = libs.tempopulsar(parfile = parfiles[0],
                           timfile = timfiles[0], maxobs=50000)
    tempopsr.append(psr)

In [12]:
# remove all noise from the pulsars
# add in EFAC noise
for psr in tempopsr:
    LT.make_ideal(psr)
    LT.add_efac(psr, efac=1.0)
    # psr.fit()
# LT.createGWB(tempopsr, Amp=2e-15, gam=13./3.)

In [11]:
for psr in tempopsr:
    print(psr.name)
    rednoise_toadd = noise_params[str(psr.name) + '_log10_A']
    # print(psr.name)
    gamma = noise_params[str(psr.name) + '_gamma']
    LT.add_rednoise(psr, 10**rednoise_toadd, gamma, components=30)
    psr.fit()

B1855+09
B1937+21
J0030+0451
J0613-0200
J1012+5307
J1024-0719
J1455-3330
J1600-3053
J1614-2230
J1640+2224
J1643-1224
J1713+0747
J1738+0333
J1741+1351
J1744-1134
J1903+0327
J1909-3744
J1910+1256
J1918-0642
J2010-1323
J2145-0750
J2317+1439


In [12]:
# create and add GWB to all the pulsars
LT.createGWB(tempopsr, Amp=3e-15, gam=13./3.)
for psr in tempopsr:
    psr.fit()

In [13]:
# save the pulsars
pardir = './fakes_gwb3/par/'
timdir = './fakes_gwb3/tim/'
if not os.path.exists(pardir):
    os.makedirs(pardir)
    os.makedirs(timdir)


for psr in tempopsr:
    psr.savepar(pardir + psr.name + '-sim.par')
    psr.savetim(timdir + psr.name + '-sim.tim')
    T.purgetim(timdir + psr.name + '-sim.tim')  # fix the tim files